In [1]:
!nvidia-smi

Mon Feb 12 05:11:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000001:00:00.0 Off |                    0 |
| N/A   43C    P0              87W / 400W |  52657MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '4'

In [3]:
from modeling_vision import MM_LLMs, MM_LLMs_Config
from transformers import AutoModelForCausalLM, CLIPProcessor, CLIPModel,AutoModel, AutoTokenizer, AutoProcessor,AutoConfig,CLIPConfig, LlamaConfig, WhisperConfig, WhisperModel, LlamaModel, LlamaTokenizer
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
import torch
import numpy as np
from torch import nn
from streaming import LocalDataset
from typing import List

In [ ]:
MM_LLMs.register_for_auto_class()
MM_LLMs_Config.register_for_auto_class()

In [ ]:
from transformers.trainer_utils import get_last_checkpoint

latest = get_last_checkpoint('vision-qwen0.5')
latest

In [4]:
model = MM_LLMs.from_pretrained(
    latest,flash_attention = True, dtype = torch.bfloat16, torch_dtype = torch.bfloat16
)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour


In [5]:
_ = model.cuda()

In [6]:
image_processor = AutoProcessor.from_pretrained('google/siglip-base-patch16-384')
tokenizer = AutoTokenizer.from_pretrained(latest)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
model.llm.generation_config.eos_token_id = tokenizer.eos_token_id

In [8]:
from PIL import Image

def prepare_dataset(messages, images: List[str] = None):
    if images is not None:
        images = [Image.open(f).convert('RGB') for f in images]
        image_output = image_processor(images=images, return_tensors='pt')['pixel_values']
    else:
        image_output = None
    
    prompt = tokenizer.apply_chat_template(messages, tokenize = False)
    outputs = tokenizer(
                    prompt,
                    return_tensors='pt',
                    return_overflowing_tokens=False,
                    return_length=False
    )

    outputs['images'] = image_output
    outputs['image_index'] = torch.tensor([0] * len(outputs['images']))
    outputs['image_starts'] = torch.tensor([tokenizer.convert_tokens_to_ids('<image>')] * len(outputs['images']))
    return outputs

In [9]:
# !wget https://cdn.beautifulnara.net/wp-content/uploads/2017/12/10201620/Persian-cat-breed.jpg
# !wget https://www.jocooks.com/wp-content/uploads/2023/09/nasi-goreng-1-23.jpg

In [10]:
test_image = 'translated-LLaVA-Instruct-150K/filtered-llava-images/000000033471.jpg'
test_image2 = 'Persian-cat-breed.jpg'
test_image3 = 'abang-gay.png'
test_image4 = 'nasi-goreng-1-23.jpg'
images = [
    test_image,
    test_image2,
    test_image3,
    test_image4
]

In [11]:
for img in images:
    messages = [
        {'role': 'user', 'content': '<image> </image> ini gambar apa'},
    ]
    outputs = prepare_dataset(messages, images = [img])
    outputs['images'] = outputs['images'].type(model.dtype)
    for k in outputs.keys():
        if outputs[k] is not None:
            outputs[k] = outputs[k].cuda()

    with torch.no_grad():
        model_inputs = model.prepare_inputs_for_generation(**outputs)
    r = model_inputs.pop('input_ids', None)

    generate_kwargs = dict(
        model_inputs,
        max_new_tokens=300,
        top_p=0.95,
        top_k=50,
        temperature=0.1,
        do_sample=True,
        num_beams=1,
    )

    r = model.llm.generate(**generate_kwargs)
    print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


<|endoftext|><|im_start|>assistant
Ini adalah gambar bas dua tingkat yang diletakkan di jalan bandar.<|im_end|>


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


<|endoftext|><|im_start|>assistant
Ini adalah gambar kucing putih yang duduk di atas sofa hitam.<|im_end|>


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


<|endoftext|><|im_start|>assistant
Imej itu menampilkan seorang lelaki berambut panjang berdiri di hadapan meja dengan beg galas di tangannya. Dia tersenyum dan tersenyum, menunjukkan bahawa dia gembira atau berpuas hati. Lelaki itu memakai baju hitam dan seluar pendek merah, yang menambahkan lagi suasana positifnya.<|im_end|>
<|endoftext|><|im_start|>assistant
Imej ini menampilkan mangkuk putih yang diisi dengan pelbagai bahan, termasuk mi, sayur-sayuran, dan bahan-bahan lain. Terdapat beberapa lobak merah yang tersebar di seluruh mangkuk, menambah warna dan tekstur pada hidangan. Selain itu, terdapat sudu yang diletakkan di atas mangkuk, mungkin untuk menghidangkan hidangan.<|im_end|>


In [13]:
messages = [
    {'role': 'user', 'content': '<image> </image> <image> </image> apa kaitan 2 gambar ni'},
]
outputs = prepare_dataset(messages, images = [test_image2, test_image4])
outputs['images'] = outputs['images'].type(model.dtype)
for k in outputs.keys():
    if outputs[k] is not None:
        outputs[k] = outputs[k].cuda()
        
with torch.no_grad():
    model_inputs = model.prepare_inputs_for_generation(**outputs)
r = model_inputs.pop('input_ids', None)
label = model_inputs.pop('labels', None)
outputs['input_ids'].shape, model_inputs['inputs_embeds'].shape

generate_kwargs = dict(
    model_inputs,
    max_new_tokens=300,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)

r = model.llm.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


<|endoftext|><|im_start|>assistant
Tiada hubungan langsung antara gambar 1 dan gambar 2. Gambar 1 ialah imej kucing putih dengan bulu putih, manakala gambar 2 ialah gambar mangkuk makan tengah hari kacang hitam dan lobak merah yang dicincang, dengan garpu diletakkan di sebelahnya. Kedua-duanya tidak berkaitan dari segi kandungan.<|im_end|>


In [ ]:
model.push_to_hub('malaysian-Qwen1.5-0.5B-siglip-base-384-vision', organization='mesolitica', safe_serialization=False)

In [ ]:
image_processor.push_to_hub('malaysian-Qwen1.5-0.5B-siglip-base-384-vision', organization='mesolitica', safe_serialization=True)

In [ ]:
tokenizer.push_to_hub('malaysian-Qwen1.5-0.5B-siglip-base-384-vision', organization='mesolitica', safe_serialization=True)